<a href="https://colab.research.google.com/github/willyferreira/bank/blob/main/bank_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Bank Customer Churn Prediction</h1>

https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers

In [ ]:
#Author: Willy Ferreira
#Date: 28/02/2023
#Link: https://github.com/willyferreira/bank

<h3>1. Instalação e importação dos pacotes</h3>

<h4>1.1. Instalação</h4>

In [ ]:
##Processamento e análise de dados
%pip install numpy

##Biblioteca para manipulação de dados
%pip install pandas

##Criação de gráficos interativos 
%pip install plotly

##Análise rápida de dados
%pip install pandas_profiling

##Interface gráfica para Jupyter
#%pip install ipywidgets

##Criação de gráficos
%pip install matplotlib

##Criação de tabelas
%pip install chart_studio

#Criação de dashboards interativos
#%pip install dash

<h4>1.2. Importação</h4>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot

# import chart_studio.plotly as py
# import plotly.figure_factory as ff

#import dash
#from dash import Dash, html, dcc
# 
#from pandas_profiling import ProfileReport
#import ipywidgets

1.2.1. Integrando o Kaggle com o Google Colab

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"willyferreira","key":"a49c6f93ab9dc09260b30c1deef78b34"}'}

<h3>2. Importação do dataset e demais tratamentos</h3>

In [ ]:
df_original = pd.read_csv(
    filepath_or_buffer = 'churn.csv', #arquivo
    sep = ',', #separação dos valores
    header = 0, #o meu arquivo tem um cabeçalho
    )

#mostra os 5 primeiros registros
df_original.head() 

#deletando a coluna RowNumber
df_original.drop(['RowNumber'], axis = 1, inplace = True)

#mostra os 5 primeiros registros novamente
df_original.head() 

In [ ]:
df = df_original.rename(
    columns={
        'CustomerId': 'id_cliente',
        'Surname': 'nome',
        'CreditScore': 'score',
        'Geography': 'pais',
        'Gender': 'genero',
        'Age': 'idade',
        'Tenure':'tempo_cliente',
        'Balance': 'saldo_cc',
        'NumOfProducts': 'qtd_produtos',
        'HasCrCard': 'cartao',
        'IsActiveMember': 'cliente_ativo',
        'EstimatedSalary': 'salario_estimado',
        'Exited': 'churn'
    }
)

df.head()

<h3>3. Análise Exploratória</h3>

In [ ]:
#Informações sobre o data frame
df.info()

O dataset possui 10.000 registros e nenhum dado nulo

<h4>3.1. Análise Exploratória - Score de Crédito</h4>

In [ ]:
df.score.describe()

In [ ]:
#######Gráfico - Score de Crédito########
#Documentação
#https://plotly.com/python/box-plots/
#https://plotly.com/python/reference/layout/
#https://plotly.com/python/reference/box/
#
fig_score = go.Figure()

#Definição do tipo e dos dados do gráfico
fig_score.add_box(x = df['score'], name = 'Score')

#Definições de layout do gráfico
#Título
fig_score.update_layout(title_text = 'Score de Crédito', #Título
                        title_font_family = 'Helvetica', #Fonte
                        title_font_color = 'black', #Cor do título
                        title_font_size = 18 #Tamanho da fonte
                        )    

fig_score.update_traces(orientation = 'h', #orientação
                        opacity = 0.8, #opacidade
                        marker_color = 'black', #cor do boxplot
                        marker_outliercolor = 'red', #cor dos outliers
                        notched = True, #inserir notch
                        notchwidth = 0.25 #tamanho do notch
                        )

#Fonte do gráfico
fig_score.update_layout(font_family = 'Helvetica')
#Tamanho do gráfico
fig_score.update_layout(autosize = True)#height = 400, width = 600)

fig_score.show()

In [ ]:
#O score de crédito mínimo observado foi de 350 e o máximo de 850. 
#A média do score de crédito dos clientes é de 650.
#O desvio padrão foi de 96, mostrando a diversidade do conjunto de dados.

<h4>3.2. Análise Exploratória - País</h4>

In [ ]:
#Criando dataframe só com informações de país
df_pais = df.filter(['pais'], axis = 1)
#obtendo informações sobre o dataframe
df_pais.describe()
#https://sparkbyexamples.com/pandas/pandas-create-new-dataframe-by-selecting-specific-columns/

In [ ]:
df_pais.info()

In [ ]:
#agrupando os registros por país
df_pais = df_pais.groupby('pais').size()
df_pais.info()
#https://acervolima.com/pandas-groupby-contar-ocorrencias-na-coluna/
#https://www.kdnuggets.com/2022/11/4-ways-rename-pandas-columns.html

In [ ]:
df_pais = df_pais.rename(columns = {0 : 'qtd'})

In [ ]:
#https://sparkbyexamples.com/pandas/pandas-percentage-total-with-groupby/

df_pais['%'] = 100 * df_pais['pais']

In [ ]:
fig_pais = go.Figure()

fig_pais.add_trace(go.Pie(labels = df['pais'], name = 'País'))

fig_pais.update_traces(hole = .4, hoverinfo = 'label+percent')

fig_pais.update_layout(title_text = 'Clientes por país',
                       title_font_family = 'Helvetica', #Fonte
                       title_font_color = 'black', #Cor do título
                       title_font_size = 18 #Tamanho da fonte)
                       )

In [ ]:
#Gráfico - Clientes por país
fig_pais = px.pie(
    data_frame = df, 
    names = 'pais',
    title = 'Clientes por País',
    color = 'pais',
    color_discrete_map = {'France': 'cyan',
                          'Germany': 'royalblue',
                          'Spain': 'darkblue'},
    hole = .4
                )
fig_pais

In [ ]:
df.idade.describe()

In [ ]:
#Gráfico - Idade
fig_idade = px.box(
    data_frame = df, 
    x = 'idade',
    title = 'Idade'
    #,
    #notched = False,
    #hover_data = ['churn']
    )
fig_idade

In [ ]:
df.idade_abertura_cc.describe()

In [ ]:
#Gráfico - Tempo de abertura da conta
fig_idade_abertura_cc= px.box(
    data_frame = df, 
    x = 'idade_abertura_cc',
    title = 'Tempo de abertura da conta'
    )
fig_idade_abertura_cc

In [ ]:
df.saldo_cc.describe()

In [ ]:
#Gráfico - Saldo em Conta Corrente
fig_saldo_cc= px.box(
    data_frame = df, 
    x = 'saldo_cc',
    title = 'Saldo em Conta Corrente'
    )
fig_saldo_cc

In [ ]:
df.qtd_produtos.describe()

In [ ]:
df.qtd_produtos.value_counts()

In [ ]:
#Gráfico - Quantidade de Produtos
fig_qtd_produtos= px.box(
    data_frame = df, 
    x = 'qtd_produtos',
    title = 'Quantidade de Produtos'
    )
fig_qtd_produtos

In [ ]:
#Gráfico - Clientes por quantidade de produtos
fig_qtd_produtos_pie = px.pie(
    data_frame = df, 
    names = 'qtd_produtos',
    title = 'Clientes por quantidade de produtos',
    color = 'qtd_produtos',
    color_discrete_map = {1: 'cyan',
                          2: 'royalblue',
                          3: 'darkblue',
                          4: 'lightblue'},
    hole = .5
                )
fig_qtd_produtos_pie

Correlações

In [ ]:
df_corr = df.corr(method = 'pearson')

In [ ]:
import plotly.graph_objects as go

fig_corr = go.Figure(
    data = go.Heatmap(
        z = df_corr,
        x = df_corr.columns,
        y = df_corr.columns
    )
)

fig_corr.show()

In [ ]:
fig_corr_arred = px.imshow(
    round(df_corr, 2),
    text_auto = True,
    aspect = 'auto'
)

fig_corr_arred.show()

In [ ]:
#profile = ProfileReport(df_bank)
#profile.to_widgets()
#profile.to_file('bank_churn.html')
#profile.to_notebook_iframe()

In [ ]:
df['pais'].unique()
#df_bank['pais'].nunique()

In [ ]:
#ver as colunas do data frame
df.columns
#ver a quantidade de linhas
df.index
#mostrar estatísticas rápidas do data frame
df.describe()
#pivotar o data frame
df.T
#monstrando o data frame ordenado pelas colunas crescentes
df.sort_index(axis = 1, ascending = True)
#mostrando o data frame ordenado por uma coluna
df.sort_values(by = 'churn')
#mostrando 5 primeiras linhas e 2 colunas específicas
df.loc[0:5:, ['ativo', 'idade']]
#mostrar 3 primeiras linhas e todas as colunas
df.iloc[0:3, :]
#mostrar uma posição específica
df.iloc[0]



In [ ]:
app = Dash(__name__)

fig = px.box(data_frame = df, x = 'score_de_credito')

app.layout = html.Div(children = [
        html.H1(children = 'Score de Crédito'),
    
dcc.Graph(id = 'example-graph', figure = fig)
]
)
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# #Leitura do arquivo CSV
# df_original = pd.read_csv(filepath_or_buffer = 'Bank Customer Churn Prediction.csv', sep = ',', header = 0, index_col = False)

# #Alteração do nome das colunas
# df = df_original.rename(columns = 
#             {'customer_id': 'id_cliente',
#              'credit_score': 'score_de_credito',
#              'country': 'pais',
#              'gender': 'genero',
#              'age': 'idade',
#              'tenure': 'idade_abertura_cc',
#              'balance': 'saldo_cc',
#              'products_number': 'qtd_produtos',
#              'credit_card': 'cartao_de_credito',
#              'active_member': 'ativo',
#              'estimated_salary': 'salario_estimado'})